In [1]:
import torch 
import pandas as pd 
import numpy as np
import torch.nn as nn


# **Building Dataset**

In [2]:
class Moadataset():
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets 
    
    def __len__(self):
        # if numpy array
        return self.features.shape[0]
    
    def __getitem__(self,item):
        return {
            'x': torch.tensor(self.features[item, :], dtype=torch.float), # will convert each row into tensor, 
            'y': torch.tensor(self.targets[item, :], dtype=torch.float)
        }
        

In [3]:
!pip install iterative-stratification

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold


In [4]:
df = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
df.loc[:, 'kfold'] = -1
df = df.sample(frac=1).reset_index(drop = True)

targets = df.drop("sig_id", axis=1).values
mskf = MultilabelStratifiedKFold(n_splits = 5)

for fold, (trn, val) in enumerate(mskf.split(X=df, y=targets)):
    df.loc[val, 'kfold'] = fold

df.to_csv('new_df', index = False)



In [5]:
df = pd.read_csv('../input/lish-moa/train_features.csv')
# let's remove cat col, so that I don't need to transform them
df = df.drop(['cp_type', 'cp_time', 'cp_dose'], axis = 1)

targets_df = pd.read_csv('new_df') 

feature_columns = df.drop('sig_id', axis=1).columns
target_columns = targets_df.drop(['sig_id', 'kfold'], axis=1).columns


# let's merge them 
df = df.merge(targets_df, on='sig_id', how= 'left')
# print(df.head())

# splitting data 
train_df = df[df.kfold != fold].reset_index(drop=True)
valid_df = df[df.kfold == fold].reset_index(drop=True)

# train test split
x_train = train_df[feature_columns].to_numpy()
y_train = train_df[target_columns].to_numpy()

x_valid = valid_df[feature_columns].to_numpy()
y_valid = valid_df[target_columns].to_numpy()

# let's create dataLoader
train_dataset = Moadataset(features=x_train, targets=y_train)
valid_dataset = Moadataset(features=x_valid, targets=y_valid)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1024, num_workers=4, shuffle = True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=1024, num_workers=4, shuffle = True)







/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# Engine Class


In [6]:
class Engine:
    def __init__(self, model, optimizer, device):
        self.model = model
        self.device = device 
        self.optimizer = optimizer
        
        
    @staticmethod
    def loss_fn(targets, outputs):
        return torch.nn.BCEWithLogitsLoss()(outputs, targets)
    
    def train(self, data_loader):
        self.model.train()
        final_loss = 0
        for data in data_loader:
            self.optimizer.zero_grad()
            input = data['x'].to(self.device)
            targets = data['y'].to(self.device)
            outputs = self.model(input)
            loss = self.loss_fn(targets, outputs)
            loss.backward()
            self.optimizer.step()
            final_loss += loss.item()
        return final_loss/len(data_loader)
    
    def evaluate(self, data_loader):
        self.model.eval()
        final_loss = 0
        for data in data_loader:
#             self.optimizer.zero_grad()
            input = data['x'].to(self.device)
            targets = data['y'].to(self.device)
            outputs = self.model(input)
            loss = self.loss_fn(targets, outputs)
#             loss.backward()
#             self.optimzer.step()
            final_loss += loss.item()
        return final_loss/len(data_loader)
    
   
    
    

# 

# **Model**

In [7]:
class Model(nn.Module):
    def __init__(self, nfeatures, ntargets, nlayers, hidden_size, dropout):
        super().__init__()
        layers = []
        for _ in range(nlayers):
            if len(layers)==0:
                layers.append(nn.Linear(nfeatures, hidden_size))
                layers.append(nn.BatchNorm1d(hidden_size))
                layers.append(nn.Dropout(dropout))
                layers.append(nn.ReLU())
            else:
                layers.append(nn.Linear(hidden_size, hidden_size))
                layers.append(nn.BatchNorm1d(hidden_size))
                layers.append(nn.Dropout(dropout))
                layers.append(nn.ReLU())
        layers.append(nn.Linear(hidden_size, ntargets))
        
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.model(x)
    
    

In [8]:
x_train.shape
y_train.shape


(19051, 206)

In [9]:
DEVICE = 'cuda'
EPOCHS = 20

# **Trainer**

In [10]:
#  model = Model(
#     nfeatures = x_train.shape[1],
#     ntargets = y_train.shape[1] , 
#     nlayers = 2, 
#     hidden_size = 128,
#     dropout = 0.3
#     )


def trainer(save_model, params): 
    # if you want to tune model parameter, then use them as parameters
    model = Model(
        nfeatures = x_train.shape[1],
        ntargets = y_train.shape[1] , 
        nlayers = params['num_layers'], 
        hidden_size = params['hidden_size'],
        dropout = params['dropout']
        )

    
    model.to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])
    save_model = save_model
    eng = Engine(model, optimizer, DEVICE)
    best_loss = np.inf
    early_stopping_iter = 10
    early_stopping_counter = 0

    for epoch in range(EPOCHS):
        train_loss = eng.train(train_loader)
        valid_loss = eng.evaluate(valid_loader)
        print(f'{epoch}, Training Loss = {train_loss}, Validation Loss = {valid_loss}')
        if valid_loss < best_loss:
            best_loss = valid_loss
            if save_model:
                torch.save(model.state_dict(), f'model_{fold}.min')
        else:
            early_stopping_counter +=1 
    
        if early_stopping_counter > early_stopping_iter:
            print('Need to stop!')
            break
    return best_loss
    
# trainer()
    

# Opimization by ***OPTUNA***

In [11]:
import optuna

In [12]:
def objective(trial):
    # every time when object is called these params are created 
    params = {
        'num_layers': trial.suggest_int('num_layer', 1, 7),
        'hidden_size': trial.suggest_int('hidden_size', 16, 2048), 
        'dropout': trial.suggest_float('dropout', 0.1, 0.7),
        'learning_rate': trial.suggest_float('learning_rate', 1e-6, 1e-3, log=True)        
    }
    
    best_loss = trainer(save_model=False, params=params)
    
    return best_loss

#     all_losses = [] 
#     for f_ in range(1):
#         # we need to add params into trainer parameter
#         temp_loss = trainer(save_model = False, params=params)
#         all_losses.append(temp_loss)
#     return np.mean(all_losses)
        

In [13]:
study = optuna.create_study(direction = 'minimize') # direction, minimizing the loss, (or maximizing the Profit if do)
study.optimize(objective, n_trials=3)


print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))



[I 2022-10-02 13:57:07,975] A new study created in memory with name: no-name-28f59893-d887-403c-ba7b-0f7beac326db


0, Training Loss = 0.7189988682144567, Validation Loss = 0.6872542381286622
1, Training Loss = 0.7055709079692238, Validation Loss = 0.6838765859603881
2, Training Loss = 0.6922811081534938, Validation Loss = 0.6805537104606628
3, Training Loss = 0.6792391475878263, Validation Loss = 0.677022910118103
4, Training Loss = 0.6662536890883195, Validation Loss = 0.672495698928833
5, Training Loss = 0.6533696243637487, Validation Loss = 0.6675014853477478
6, Training Loss = 0.6401826551086024, Validation Loss = 0.661819314956665
7, Training Loss = 0.6267094706234179, Validation Loss = 0.6551522254943848
8, Training Loss = 0.613229513168335, Validation Loss = 0.6477950930595398
9, Training Loss = 0.5995293510587592, Validation Loss = 0.6396161198616028
10, Training Loss = 0.5859962043009306, Validation Loss = 0.6314927458763122
11, Training Loss = 0.5723913594296104, Validation Loss = 0.6223288536071777
12, Training Loss = 0.5591604646883512, Validation Loss = 0.6131049513816833
13, Training 

[I 2022-10-02 13:57:42,355] Trial 0 finished with value: 0.5460810542106629 and parameters: {'num_layer': 7, 'hidden_size': 1940, 'dropout': 0.3774623019864146, 'learning_rate': 2.168595742915985e-06}. Best is trial 0 with value: 0.5460810542106629.


19, Training Loss = 0.4732854632954848, Validation Loss = 0.5460810542106629
0, Training Loss = 0.28581757098436356, Validation Loss = 0.08135872334241867
1, Training Loss = 0.04181105349408953, Validation Loss = 0.028835994750261308
2, Training Loss = 0.025175466741386213, Validation Loss = 0.022620395198464392
3, Training Loss = 0.02203349886756194, Validation Loss = 0.0204740971326828
4, Training Loss = 0.0205446652283794, Validation Loss = 0.01950938142836094
5, Training Loss = 0.01963704246047296, Validation Loss = 0.01894281469285488
6, Training Loss = 0.01891429526241202, Validation Loss = 0.018336814641952515
7, Training Loss = 0.0183315880008434, Validation Loss = 0.01796785406768322
8, Training Loss = 0.017859198937290592, Validation Loss = 0.017787068709731103
9, Training Loss = 0.01744995570104373, Validation Loss = 0.01745774783194065
10, Training Loss = 0.017065516329909627, Validation Loss = 0.017284632474184037
11, Training Loss = 0.01675003188613214, Validation Loss = 

[I 2022-10-02 13:58:06,870] Trial 1 finished with value: 0.016178210824728013 and parameters: {'num_layer': 1, 'hidden_size': 714, 'dropout': 0.593033091832533, 'learning_rate': 0.000723125746087448}. Best is trial 1 with value: 0.016178210824728013.


19, Training Loss = 0.01463868174897997, Validation Loss = 0.016178210824728013
0, Training Loss = 0.4603151851578763, Validation Loss = 0.3854027628898621
1, Training Loss = 0.14178135010756945, Validation Loss = 0.061428947001695634
2, Training Loss = 0.05127113881079774, Validation Loss = 0.025505777448415756
3, Training Loss = 0.03180726588164505, Validation Loss = 0.025711993128061293
4, Training Loss = 0.026972919310394087, Validation Loss = 0.02515690103173256
5, Training Loss = 0.024810072426733217, Validation Loss = 0.02386394664645195
6, Training Loss = 0.023447635436528606, Validation Loss = 0.022644463554024698
7, Training Loss = 0.022512738641939666, Validation Loss = 0.021982625126838684
8, Training Loss = 0.021835161078917354, Validation Loss = 0.02145630493760109
9, Training Loss = 0.021339028191409613, Validation Loss = 0.020905207470059396
10, Training Loss = 0.020945766744645017, Validation Loss = 0.020675623789429665
11, Training Loss = 0.020642005691402836, Validat

[I 2022-10-02 13:58:32,569] Trial 2 finished with value: 0.01921963021159172 and parameters: {'num_layer': 7, 'hidden_size': 259, 'dropout': 0.11008616593844459, 'learning_rate': 0.0007582451772237903}. Best is trial 1 with value: 0.016178210824728013.


19, Training Loss = 0.019454110217721837, Validation Loss = 0.01921963021159172
Best trial:
  Value:  0.016178210824728013
  Params: 
    num_layer: 1
    hidden_size: 714
    dropout: 0.593033091832533
    learning_rate: 0.000723125746087448
